In [249]:
import wget
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output

In [7]:
wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

'spacex_launch_dash.csv'

In [8]:
wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py")

'spacex_dash_app.py'

In [11]:
from jupyter_dash import JupyterDash

app = JupyterDash(__name__)

In [24]:
spacex_df.rename(columns={"Launch Site": "Launch_Site"})
spacex_df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [16]:
spacex_df.drop(['Unnamed: 0'], axis=1, inplace=True) 
spacex_df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [30]:
all(isinstance(column, str) for column in spacex_df.columns) #confirm all columns are strings

True

In [181]:
launch_list = spacex_df["Launch Site"].unique().tolist()  #add unique sites to a list object
launch_list.insert(0,"All Sites") #add All Sites to the list of launch sites with 0 index position
for i in launch_list: #check the for loop works and use for the options.
    print(i)

All Sites
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [267]:
# Import required libraries
import pandas as pd
import dash
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


# Create a dash application
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                #create division for instruction to use down menu
                                #Add the dropdown dcc.Dropdown(id='site-dropdown',...)
                                html.Div(['Select Launch Site:', dcc.Dropdown(id='site-dropdown',
                                                     options=[{'label': i, 'value': i} for i in launch_list],
                                                     value='All Sites',
                                                     placeholder="Select a Launch Site here:",
                                                     searchable=True),
                                            ]
                                        ),

                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success_pie_chart')),
                                
                                html.Br(),

                                #there is a problem with this square bracker and there not being a list
                                html.P("Payload range (Kg):"),
                              
                                # TASK 3: Add a slider to select payload range https://www.youtube.com/watch?v=Zvz2LpziQAs
                                #dcc.RangeSlider(id='payload-slider',...)
                                html.Div(dcc.RangeSlider(id='payload-slider',
                                    marks={
                                        0: '0',
                                        2500: '2500',
                                        5000: '5000',
                                        7500: '7500',
                                        10000: '10000',
                                    },
                                    step=1000,
                                    min=0,
                                    max=10000,
                                    value=[0, 2000],
                                    dots=True
                                    
                                )),
            

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ]
                     )

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(Output(component_id='success_pie_chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
             

def get_pie_chart (entered_site):
    filtered_df = spacex_df
    if entered_site == 'All Sites':
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Total Success Launches By Site')
        return fig

    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site].groupby(['Launch Site', 'class']). \
        size().reset_index(name='class count')
        title = f"Total Successful Launches for site {entered_site}" #inserts selected site into title
        fig = px.pie(filtered_df,values='class count', names='class', title=title)
        return fig
    
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output    
 
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id="payload-slider", component_property="value")])
             

def get_scatter (site, slider): #inputs are to define the site dropdown and the slider use.

    slider_df=spacex_df[(spacex_df['Payload Mass (kg)'] > slider[0]) & (spacex_df['Payload Mass (kg)'] < slider[1])]
    
    if site == 'All Sites':
        fig = px.scatter(slider_df, 
            x="Payload Mass (kg)", 
            y="class", 
            color = "Booster Version Category",
            title = "Booster Payload and Success for all Launch Sites" )
        return fig

    else:
        slider_df = slider_df[slider_df['Launch Site'] == site] #filters slider results by launch site
        title = f"Payload versus success/failure for {site}"
        fig = px.scatter(
            slider_df, 
            x="Payload Mass (kg)", 
            y="class", 
            color = "Booster Version Category",
            title = f"Booster Payload and Success for {site}" )
        return fig
 



# Run the app
if __name__ == '__main__':
    # REVIEW8: Adding dev_tools_ui=False, dev_tools_props_check=False can prevent error appearing before calling callback function
    app.run_server(mode="inline", host="localhost", debug=False, dev_tools_ui=False, dev_tools_props_check=False)



127.0.0.1 - - [20/Apr/2022 23:36:18] "GET /_shutdown_ce5cf2e1-df79-49cf-a550-688a07208e0b HTTP/1.1" 200 -
 * Running on http://localhost:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Apr/2022 23:36:18] "GET /_alive_ce5cf2e1-df79-49cf-a550-688a07208e0b HTTP/1.1" 200 -


127.0.0.1 - - [20/Apr/2022 23:36:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 23:36:19] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 23:36:19] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 23:36:19] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 23:36:19] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 23:36:19] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 23:36:19] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 23:36:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 23:36:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 23:36:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2022 23:36:28] "POST /_dash-update-component HTTP/1.1" 200 -


In [204]:
#Terminate running on a local host.
if __name__ == '__main__':
    app._terminate_server_for_port("localhost", 8050)

127.0.0.1 - - [20/Apr/2022 08:42:09] "GET /_shutdown_ce5cf2e1-df79-49cf-a550-688a07208e0b HTTP/1.1" 200 -
